In [1]:
import random
import pandas as pd
directory = "C:/Users/Sergio/Documents/python projects/Dice/"

In [2]:
class Die: # 
    def __init__(self,sides): 
        self.sides = sides #sides specified as an array at initialization
        self.score = 0
        
        self.wins = 0 #how many MUs it wins/draws/loses
        self.losses = 0
        self.contests = 0
        
        self.possWins = 0 #how many possible interactions it wins/draws/loses
        self.possLosses = 0
        self.possContests = 0
        
        self.tournamentWins = 0
        
    def show(self): #print info for die
        draws = self.contests - (self.wins + self.losses)
        possDraws = self.possContests - (self.possWins + self.possLosses)
        
        winPercent = str(100*self.wins/self.contests)[0:6] + "%"
        drawPercent = str(100*draws/self.contests)[0:6] + "%"
        lossPercent = str(100*self.losses/self.contests)[0:6] + "%"
        
        possWinPercent = str(100*self.possWins/self.possContests)[0:6] + "%"
        possDrawPercent = str(100*possDraws/self.possContests)[0:6] + "%"
        possLossPercent = str(100*self.possLosses/self.possContests)[0:6] + "%"
               
        
        print(self.sides,end = "\t")
        print(str(self.wins) + "\t" + str(draws) + "\t" + str(self.losses),end = "\t")
        print(winPercent + "\t" + drawPercent + "\t" + lossPercent,end = "\t")
        
        print(str(self.possWins) + "\t" + str(possDraws) + "\t" + str(self.possLosses),end = "\t")
        print(possWinPercent + "\t" + possDrawPercent + "\t" + possLossPercent,end = "\t")
        
        print()

        
    def play(self):
        self.contests = self.contests + 1
    
    def win(self):
        self.wins = self.wins+1
        
    def lose(self):
        self.losses = self.losses+1
        
    def roll(self):
        return random.choice(self.sides)
    
    def half(self): # divides attributes by 2 since all values are doubled when calculated
        self.wins /= 2
        self.losses /= 2
        self.contests /= 2
        
        self.possWins /= 2
        self.possLosses /= 2
        self.possContests /= 2
        
    def clearMatches(self): # reset matches data
        self.wins = 0
        self.losses = 0
        self.contests = 0
        
        self.possWins = 0
        self.possLosses = 0
        self.possContests = 0
        
    def winTournament(self):
        self.tournamentWins += 1
        
    def clearTournaments(self):
        self.tournamentWins = 0

In [3]:
class AllDice:# set of dice
    def __init__(self):
        self.dice = [] # dice in set as array of dice
        self.build() # build a set of every possible die
        #self.show()
        
    def build(self):# every [possible 6-sided die with 21 total]
        for a in range(4,22):# largest number in the dice can only be between 4 and 21 ([4,4,4,3,3,3] to [21,0,0,0,0,0])
            for b in range(11):# second largest number can only be as high as 10 [11,10,0,0,0,0]
                if a<b:
                    break # if the 2nd number is greater than the 1st, quit out of loop
                for c in range(8):# 3rd number goes up to 7 [7,7,7,0,0,0]
                    if b<c:
                        break
                    for d in range(6):# 4th number goes to 5 [6,5,5,5,0,0]
                        if c<d:
                            break
                        for e in range(5):# and so on [5,4,4,4,4,0]
                            if d<e:
                                break
                            for f in range(4):# [4,4,4,3,3,3]
                                if e<f:
                                    break
                                if (a+b+c+d+e+f)==21:# only add to list if numbers add to 21
                                    self.dice.append(Die([a,b,c,d,e,f]))
                                    
    def show(self):# print data for every die in table-friendly manner
        #print(len(self.dice))
        print("sides",end = "\t")
        
        print("MU wins",end = "\t")
        print("MU draws",end = "\t")
        print("MU losses",end = "\t")
        print("MU win %",end = "\t")
        print("MU draw %",end = "\t")
        print("MU loss %",end = "\t")
        
        print("total wins",end = "\t")
        print("total draw",end = "\t")
        print("total losses",end = "\t")
        print("total win %",end = "\t")
        print("total draws %",end = "\t")
        print("total loss %")
        for i in self.dice:
            i.half()
            i.show()
            
    def clearMatches(self):
        for i in self.dice:
            i.clearMatches()
            
    def clearTournaments(self):
        for i in self.dice:
            i.clearTournaments() 
            
    def fightAll(self): # analyze all dice by comparing them to each other twice
        for i in self.dice:
            for j in self.dice:
                Contest(i,j)
                
    def limitMU(self,limit): # removes dice below a certain MU win % (not used)
        passList = []
        for i in self.dice:
            if (i.wins/i.contests) >= limit:
                passList.append(i)
            self.dice = passList
            
    def limitTotal(self,limit): # removes dice below a certain possible interaction win % (not used)
        passList = []
        for i in self.dice:
            if (i.possWins/i.possContests) >= limit:
                passList.append(i)
            self.dice = passList
            
    def removeObsolete(self): # removes obsolete dice (dice that win no MUs or only win MUs against other obsolete dice)
        passList = []
        count = 0
        
        for i in self.dice:
            if i.wins > 0:
                passList.append(i)
            else:
                count += 1
            self.dice = passList
        
        if count > 0:
            self.clearMatches()
            self.fightAll()
            self.removeObsolete()
            
    def removeNumber(self,number):
        passList = []
        for d in self.dice:
            if number not in d.sides:
                passList.append(d)
        self.dice = passList
            
    def showMUs(self,die): # print every MU within the set for a given die
        print(die.sides,end = "\t")
        print("Matchups")
        for d in self.dice:
            winner,margin = Contest(die,d)
            print(d.sides,end = "\t")
            if (winner == die) or (not winner):
                print("+" + str(margin))
            else:
                print("-" + str(margin))
                
    def saveMUs(self,die): # print every MU within the set for a given die into txt file
        MUfolder = directory + "MUs/" # MU txt folder
        
        try:
            f = open(MUfolder + str(die.sides) + ".txt", 'w')
            
            f.write("Matchup\tValue\n")
        
            for d in self.dice:
                winner,margin = Contest(die,d)
                f.write(str(d.sides) + "\t")
                if (winner == die) or (not winner):
                    f.write("+" + str(margin))
                else:
                    f.write("-" + str(margin))
                f.write("\n")
                    
        finally:
            f.close()
            
    def saveAllMUs(self):
        for d in self.dice:
            self.saveMUs(d)
            
    def tableMUs(self):
        try:
            f = open("MU Table.csv",'w')
            
            f.write("Die")
            
            for d in self.dice:
                f.write("\t" + str(d.sides))
            
            for d in self.dice:
                f.write("\n")
                f.write(str(d.sides))
                
                for e in self.dice:
                    f.write("\t")
                    winner,margin = Contest(d,e)
                    
                    if (winner == d) or (not winner):
                        f.write("+" + str(margin))
                    else:
                        f.write("-" + str(margin))
                        
        finally:
            f.close()
        

In [4]:
class Player: # a player within a competitive scene with a main die
    def __init__(self,dice):
        self.dice = dice # set of dice to choose from
        self.mainDie = random.choice(self.dice) # start with a random main die
        self.wins = 0
        self.intelligent = False
        self.attach = 5 # starting attachment value
        self.attachCap = 500
        self.scene = 0
    
    def show(self):
        print("Main: ",end = "\t")
        print(self.mainDie.sides,end = "\t")
        print(self.attach)
        
    def showWins(self):
        print("Main: ",end = "\t")
        print(self.mainDie.sides,end = "\t")
        print(self.wins)
        
    def changeMains(self): # pick a random new main
        if self.intelligent:
            self.scene.evaluateMeta()
            topDice = []
            topScore = 0
            for d in self.dice:
                if d.score >= topScore:
                    if d.score > topScore:
                        topScore = d.score
                        topDice = []
                    topDice.append(d)
            self.mainDie = random.choice(topDice)
            self.attach = 1
        else:
            self.mainDie = random.choice(self.dice)
            self.attach = 5
        
         # reset attachment
        
    def lose(self): # if a player loses a fight, their attachment goes down
        self.attach -= 1
        if self.attach < 1: # change mains if attachment reaches 0
            self.changeMains()
    
    def win(self):
        if self.attach < self.attachCap: #attachment cannot exceed 1000 (done to ensure metagame does not slow down due to inertia)
            self.attach += 1
        
    def roll(self): # roll its main die
        return self.mainDie.roll()
    
    def chooseMain(self,die):
        self.mainDie = die
        
    def winTournament(self):
        self.wins += 1
    
    def makeIntelligent(self):
        self.intelligent = True
        self.attachCap = 1

In [5]:
class Scene:# competitive scene full of players
    def __init__(self,popSize,dice):
        self.players = [] # players in scene
        self.popSize = popSize # number of players in scene
        self.dice = dice # set of dice available in scene
        self.day = 0 # current generation (1 day per tournament)
        
        for i in range(self.popSize): # populate scene
            self.players.append(Player(self.dice))
            
    def tournament(self): # each player fights 1 random other player
        bracket = self.players.copy()
        random.shuffle(bracket)
        while len(bracket) > 0:
            playerA = bracket.pop()
            playerB = bracket.pop()
            Fight(playerA,playerB)
        
            
    def show(self):
        for i in self.players:
            i.show()
            
    def countDie(self,die): # count how many players currently main a given die
        count = 0
        for p in self.players:
            if p.mainDie == die:
                count += 1
        return count
    
    def stabilize(self):
        while (len(self.players) % len(self.dice)) > 0:
            self.players.append(Player(self.dice))
        
        for i in range(len(self.players)):
            j = i
            if j >= len(self.dice):
                j -= len(self.dice)
            p = self.players[i]
            d = self.dice[j]
            p.chooseMain(d)
            
    def makePlayersIntelligent(self):
        for p in self.players:
            p.makeIntelligent()
            p.scene = self
            
    def evaluateDie(self,die):
        dieScore = 0
        for p in self.players:
            dieScore += matchups.at[str(die.sides),str(p.mainDie.sides)]
        die.score = dieScore
        return dieScore
    
    def evaluateMeta(self):
        for d in self.dice:
            dieScore = self.evaluateDie(d)
            
    def choosePlayerMain(self,die):
        for p in self.players:
            p.chooseMain(die)
            
    def runSceneSim(self,duration,interval): # run simulation
        try:
            f = open(directory + "SimulationData.csv", 'w')
            
            f.write("Day\t")
            for d in self.dice:
                f.write(str(d.sides))
                f.write("\t")
            f.write("\n")
            percent = duration / 100
            
            for i in range(duration):
#                 self.evaluateMeta()
                self.tournament()
                if self.day % interval == 0:
                    f.write(str(self.day))
                    f.write(str("\t"))
                    for d in self.dice:
                        f.write(str(self.countDie(d)))
                        f.write("\t")
                    f.write("\n")
                self.day += 1
                
                if (i % percent) == 0:
                    print(str((i/duration)*100) + "%")
        finally:        
            f.close()
            print("done!")

In [6]:
class Bracket:
    def __init__(self,bestOf,size,dice):
        self.bestOf = bestOf
        self.size = size
        self.dice = dice
        number = len(self.dice)
        self.players = []
        
        for i in range(self.size):
            self.players.append(Player(self.dice))
            
#         for i in range(number):
#             p = self.players[i]
#             d = self.dice[i]
#             p.chooseMain(d)
            
    def show(self):
        for p in self.players:
            p.showWins()
            
    def stabilize(self):
        while (len(self.players) % len(self.dice)) > 0:
            self.players.append(Player(self.dice))
        
        for i in range(len(self.players)):
            j = i
            while j >= len(self.dice):
                j -= len(self.dice)
            p = self.players[i]
            d = self.dice[j]
            p.chooseMain(d)
            
    def tournament(self): 
        bracket = self.players.copy()
        winBracket = []
        loseBracket = []
        dead = []
        random.shuffle(bracket)
        
        while len(bracket) > self.size:
            bracket.pop()
        
        winBracket, loseBracket = Split(bracket, self.bestOf)
        
        while (len(winBracket) != 1) or (len(loseBracket) != 1): 
            random.shuffle(winBracket)
            random.shuffle(loseBracket)
            
            bracket = loseBracket.copy()
            
            loseBracket, dead = Split(bracket, self.bestOf)
            
            bracket = winBracket.copy()
            winBracket, bracket = Split(bracket, self.bestOf)
            for p in bracket:
                loseBracket.append(p)

            random.shuffle(loseBracket)
            
            bracket = loseBracket.copy()
            loseBracket, dead = Split(bracket, self.bestOf)
        
        playerA = winBracket[0]
        playerB = loseBracket[0]
        
        winner, loser = Match(playerA,playerB,self.bestOf)
        
        if winner == playerB:
                winner, loser = Match(winner,loser,self.bestOf)
        
        winner.winTournament()
        winner.mainDie.winTournament()
        
    def series(self, iterations):
        for i in range(iterations):
            self.tournament()
            


In [7]:
def Split(bracket, bestOf):
    winners = []
    losers = []
    
    while len(bracket) > 0:
        playerA = bracket.pop()
        playerB = bracket.pop()
        winner,loser = Match(playerA,playerB,bestOf)
        winners.append(winner)
        losers.append(loser)
        
    return winners, losers

In [8]:
def Match(playerA,playerB,bestOf):
    
    targetScore = (bestOf+1)/2
    scoreA = 0
    scoreB = 0
    
    while (scoreA < targetScore) and (scoreB < targetScore):
        if Game(playerA,playerB) == playerA:
            scoreA += 1
        else:
            scoreB += 1
            
    if scoreA == targetScore:
        return playerA,playerB
    else:
        return playerB,playerA
    

In [9]:
def Game(playerA,playerB):
    
    rollA = playerA.roll()
    rollB = playerB.roll()
    
    while rollA == rollB:    
        rollA = playerA.roll()
        rollB = playerB.roll()
    
    if rollA>rollB:
        return playerA
    else:
        return playerB

In [10]:
def simulateSeries(dice, size):
    try:
        f = open("Tournament Series large.csv",'w')
        
        f.write("size\tBest of")
        for d in dice:
            f.write("\t")
            f.write(str(d.sides))
        f.write("\n")
        
        for s in range(9,13):
            s = 2**s
            print(s)
            
            for b in [3,5]:
                bra = Bracket(b,s,dice)
                bra.stabilize()
                bra.series(size)
                
                f.write(str(s) + "\t" + str(b))
                
                for d in bra.dice:
                    f.write("\t")
                    f.write(str(d.tournamentWins))
                    d.clearTournaments()
                f.write("\n")       
        
    finally:
        print("done!")
        f.close()

In [11]:
def Contest(dieA,dieB): # analytical comparison between 2 dice
    contests = 0
    winsA = 0
    lossA = 0
    winsB = 0
    lossB = 0
    
    dieA.play()
    dieB.play()
    
    for a in dieA.sides:
        for b in dieB.sides:
            contests = contests + 1
            if a>b:
                winsA = winsA + 1
                lossB = lossB + 1
            elif b>a:
                winsB = winsB + 1
                lossA = lossA + 1
                
    dieA.possWins += winsA
    dieA.possLosses += lossA
    dieA.possContests += contests
    
    dieB.possWins += winsB
    dieB.possLosses += lossB
    dieB.possContests += contests
    
                
    if winsA>winsB:
        dieA.win()
        dieB.lose()
        return dieA,winsA-winsB
    elif winsB>winsA:
        dieA.lose()
        dieB.win()
        return dieB,winsB-winsA
    else:
        return False,0

In [12]:
def Fight(playerA,playerB): # both players compete by rolling their main dice
    rollA = playerA.roll()
    rollB = playerB.roll()
    
    if rollA>rollB:
        playerA.win()
        playerB.lose()
    elif rollB>rollA:
        playerA.lose()
        playerB.win()

In [13]:
matchups = pd.read_csv("MU Table.csv",delimiter = "\t")
matchups = matchups.set_index("Die")

In [14]:
everyDie = AllDice()
dice = AllDice() # generate dice

# dice.fightAll()
# dice.removeObsolete()
#dice.show() # set of non-obsolete dice

In [15]:
simulateSeries(everyDie.dice,10000)

512
1024
2048
4096
done!


In [ ]:
# compScene = Scene(500,dice.dice) # generate scene of players
# compScene.stabilize()
# compScene.choosePlayerMain(dice.dice[93])
# compScene.makePlayersIntelligent()

In [ ]:
# compScene.runSceneSim(20,1)

In [ ]:
df = pd.read_csv("SimulationData.csv",delimiter = "\t")
df = df.dropna(axis='columns')
df = df.set_index('Day')


# df_max = df.cummax()
# df_max = df_max.drop(range(499))
# df_max.rename(index)

# df_diff = df.diff()

display(df)

In [ ]:
plt = df.plot(figsize=[20,15])

In [ ]:
display(plt)

In [ ]:
compScene.show()